In [1]:
import pandas as pd
import json

In [16]:
info_df   = pd.read_csv('Data/superhero_info.csv')
powers_df =pd.read_csv('Data/superhero_powers.csv')

In [17]:
print(info_df.head())
print(powers_df.head())

                 Hero|Publisher Gender               Race Alignment  \
0          A-Bomb|Marvel Comics   Male              Human      good   
1  Abe Sapien|Dark Horse Comics   Male      Icthyo Sapien      good   
2            Abin Sur|DC Comics   Male            Ungaran      good   
3     Abomination|Marvel Comics   Male  Human / Radiation       bad   
4   Absorbing Man|Marvel Comics   Male              Human       bad   

  Hair color Eye color Skin color  \
0    No Hair    yellow    Unknown   
1    No Hair      blue       blue   
2    No Hair      blue        red   
3    No Hair     green    Unknown   
4    No Hair      blue    Unknown   

                                   Measurements  
0  {'Height': '203.0 cm', 'Weight': '441.0 kg'}  
1   {'Height': '191.0 cm', 'Weight': '65.0 kg'}  
2   {'Height': '185.0 cm', 'Weight': '90.0 kg'}  
3  {'Height': '203.0 cm', 'Weight': '441.0 kg'}  
4  {'Height': '193.0 cm', 'Weight': '122.0 kg'}  
    hero_names                                    

In [18]:
# split the Name column into two columns using pd.Series.str.split()
info_df[['Hero', 'Publisher']] = info_df['Hero|Publisher'].str.split('|', expand=True)
info_df.drop(['Hero|Publisher'], axis=1, inplace=True)

In [19]:
print(info_df.head())

  Gender               Race Alignment Hair color Eye color Skin color  \
0   Male              Human      good    No Hair    yellow    Unknown   
1   Male      Icthyo Sapien      good    No Hair      blue       blue   
2   Male            Ungaran      good    No Hair      blue        red   
3   Male  Human / Radiation       bad    No Hair     green    Unknown   
4   Male              Human       bad    No Hair      blue    Unknown   

                                   Measurements           Hero  \
0  {'Height': '203.0 cm', 'Weight': '441.0 kg'}         A-Bomb   
1   {'Height': '191.0 cm', 'Weight': '65.0 kg'}     Abe Sapien   
2   {'Height': '185.0 cm', 'Weight': '90.0 kg'}       Abin Sur   
3  {'Height': '203.0 cm', 'Weight': '441.0 kg'}    Abomination   
4  {'Height': '193.0 cm', 'Weight': '122.0 kg'}  Absorbing Man   

           Publisher  
0      Marvel Comics  
1  Dark Horse Comics  
2          DC Comics  
3      Marvel Comics  
4      Marvel Comics  


In [20]:
powers_df.rename(columns = {'hero_names':'Hero'}, inplace = True)

In [21]:
Hero_df = pd.merge(info_df, powers_df, on="Hero", how="inner")

In [22]:
Hero_df.head

<bound method NDFrame.head of      Gender               Race Alignment        Hair color Eye color  \
0      Male              Human      good           No Hair    yellow   
1      Male      Icthyo Sapien      good           No Hair      blue   
2      Male            Ungaran      good           No Hair      blue   
3      Male  Human / Radiation       bad           No Hair     green   
4      Male              Human       bad           No Hair      blue   
..      ...                ...       ...               ...       ...   
458    Male              Human      good             Blond      blue   
459  Female              Human      good  Strawberry Blond      blue   
460    Male     Yoda's species      good             White     brown   
461  Female              Human      good             Black      blue   
462    Male            Unknown       bad             Brown       red   

    Skin color                                  Measurements             Hero  \
0      Unknown  {'Height

In [23]:
Measurements = Hero_df['Measurements'].str.replace("'",'"').apply(json.loads).apply(pd.Series)
Hero_df['Height'] = pd.to_numeric(Measurements['Height'].str.replace(' cm',''))
Hero_df['Weight'] = pd.to_numeric(Measurements['Weight'].str.replace(' kg',''))
Hero_df.drop(['Measurements'], axis=1, inplace=True)
Hero_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Powers,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,"Accelerated Healing,Durability,Longevity,Super...",203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,"Agility,Accelerated Healing,Cold Resistance,Du...",191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,Lantern Power Ring,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,"Accelerated Healing,Intelligence,Super Strengt...",203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,"Cold Resistance,Durability,Energy Absorption,S...",193.0,122.0


In [24]:
# Encoding the Powers
Hero_df = pd.concat([Hero_df.drop(['Powers'], axis=1), Hero_df['Powers'].str.get_dummies(sep=",")],axis=1)
Hero_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,0,0,0,0,0,0,0,0,0,0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,0,0,0,0,0,0,1,0,0,0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,0,0,0,0,0,0,0,0,0,0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,0,0,0,0,0,0,0,0,0,0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#1. Compare the average weight of super powers who have Super Speed to those who do not.
Hero_df.groupby('Super Speed')['Height'].mean()

Super Speed
0    186.376226
1    189.444444
Name: Height, dtype: float64

In [26]:
#2. What is the average height of heroes for each publisher?
Hero_df.groupby('Publisher')['Height'].mean()

Publisher
DC Comics            181.923913
Dark Horse Comics    176.909091
George Lucas         159.600000
Image Comics         211.000000
Marvel Comics        191.546128
Shueisha             171.500000
Star Trek            181.500000
Team Epic TV         180.750000
Unknown              178.000000
Name: Height, dtype: float64